# Neo4j GraphRAG Retriever Notebook

This notebook demonstrates how to use various retrievers and Cypher patterns with Neo4j GraphRAG for asset manager and cybersecurity risk retrieval.


In [ ]:
import os
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever, Text2CypherRetriever
from neo4j_graphrag.generation import GraphRAG
from dotenv import load_dotenv
import pandas as pd

from IPython.display import display, HTML

import textwrap

from IPython.core.display import HTML
display(HTML("<style>.output_area pre {white-space: pre-wrap; word-break: break-word;}</style>"))

# Load environment variables
load_dotenv()
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USER = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))


In [39]:
# --- Initialize LLM and Embedder ---
###model_params = {"response_format": {"type": "json_object"},  # use json_object formatting for best results
###                "temperature": 0  # turning temperature down for more deterministic results
###                }

llm = OpenAILLM(model_name='gpt-4o', api_key=openai_api_key)
embedder = OpenAIEmbeddings(api_key=openai_api_key)

In [64]:
# --- Initialize Vector Retriever ---

query = "What are the risks that Apple faces?"

vector_retriever = VectorRetriever(
    driver=driver,
    index_name='chunkEmbeddings',
    embedder=embedder,
    return_properties=['text'])

In [65]:
    # --- Simple Vector Search Diagnostic ---

result = vector_retriever.search(query_text=query, top_k=10)
result_table=pd.DataFrame([(item.metadata['score'], item.content[10:80], item.metadata['id']) for item in vector_result.items], columns=['Score', 'Content', 'ID'])
result_table

,Score,Content,ID
0,0.917770,"the Company to potential liabilities, increase...",4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1824
1,0.917770,"the Company to potential liabilities, increase...",4:f86fa30c-6448-4779-bdfc-2f048de6bb86:4216
2,0.914780,"relief against the Company, and has\nfrom time...",4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1823
3,0.914780,"relief against the Company, and has\nfrom time...",4:f86fa30c-6448-4779-bdfc-2f048de6bb86:4215
4,0.913116,in both\nfrequency and sophistication with in...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1817
5,0.913116,in both\nfrequency and sophistication with in...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:4207
6,0.910446,interruptions can harm or disrupt internationa...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:4187
7,0.910446,interruptions can harm or disrupt internationa...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1802
8,0.909439,impact the Company\'s net sales to its indirec...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:4185
9,0.909439,impact the Company\'s net sales to its indirec...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1800


In [ ]:
rag = GraphRAG(
    llm=llm,
    retriever=vector_retriever
)
#print(textwrap.fill(rag.search(query).answer, width=80))
print(rag.search(query).answer)

In [68]:
# --- VectorCypherRetriever Example: Detailed Search with Context ---

query = "What are the top risk factors that Apple faces?"

detail_context_query = """
    with node
    MATCH (node)-[:FROM_DOCUMENT]-(doc:Document)-[:FILED]-(company:Company)-[:FACES_RISK]-(r:RiskFactor)
    RETURN node.text AS context,  collect(DISTINCT r) AS risks
    LIMIT 10
"""
vector_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name='chunkEmbeddings',
    embedder=embedder,
    retrieval_query=detail_context_query
)

result = vector_cypher_retriever.search(query_text=query, top_k=500)
result_table=pd.DataFrame([(item.metadata['score'], item.content[10:80], item.metadata['id']) for item in result.items], columns=['Score', 'Content', 'ID'])
result_table

,Score,Content,ID


In [62]:
rag = GraphRAG(llm=llm, retriever=vector_cypher_retriever)
print(textwrap.fill(rag.search(query).answer, width=80))
#print(rag.search(query).answer)

NameError: name 'vector_cypher_retriever' is not defined

In [54]:
response.retriever_result

RetrieverResult(items=[], metadata={'__retriever': 'VectorCypherRetriever'})

In [49]:
some_var =detail_context_cypher_retriever.get_search_results(query_text=query,top_k=3)
some_var

RawSearchResult(records=[], metadata=None)

In [98]:
# --- Question Answering ---
print("\n--- Question Answering ---")
query = "Compare Apple's and Microsoft's market positions"
compare_query = """
    MATCH (c1:Company {name: 'Apple'})-[:HAS_METRIC]-(m1:FinancialMetric)
    MATCH (c2:Company {name: 'Microsoft'})-[:HAS_METRIC]-(m2:FinancialMetric)
    RETURN c1, c2, collect(m1) as apple_metrics, collect(m2) as microsoft_metrics
"""
compare_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name="chunkEmbeddings",
    embedder=embedder,
    retrieval_query=compare_query
)
compare_cypher_rag = GraphRAG(llm=llm, retriever=compare_cypher_retriever)
response = compare_cypher_rag.search(query, retriever_config={'top_k': 3})
print("\nComparison Response:")
print(response.answer)
response



--- Question Answering ---

Comparison Response:
As of the latest data available up to October 2023, Apple and Microsoft both hold strong positions in the technology sector, each excelling in different areas of the market.

Apple:
1. **Consumer Electronics Dominance**: Apple is a leader in consumer electronics, particularly with its iPhone, iPad, and Mac product lines. The company has a loyal customer base and is known for its premium pricing strategy, which contributes to high profit margins.

2. **Ecosystem and Services**: Apple's ecosystem, which includes the App Store, iCloud, Apple Music, and other services, creates a seamless user experience and generates a substantial portion of its revenue. The services segment has been growing steadily, providing recurring revenue streams.

3. **Innovation**: Apple is renowned for its innovation in hardware and design, consistently setting trends within the market. This is evidenced by its development of new product lines such as Apple Watch 

RagResultModel(answer="As of the latest data available up to October 2023, Apple and Microsoft both hold strong positions in the technology sector, each excelling in different areas of the market.\n\nApple:\n1. **Consumer Electronics Dominance**: Apple is a leader in consumer electronics, particularly with its iPhone, iPad, and Mac product lines. The company has a loyal customer base and is known for its premium pricing strategy, which contributes to high profit margins.\n\n2. **Ecosystem and Services**: Apple's ecosystem, which includes the App Store, iCloud, Apple Music, and other services, creates a seamless user experience and generates a substantial portion of its revenue. The services segment has been growing steadily, providing recurring revenue streams.\n\n3. **Innovation**: Apple is renowned for its innovation in hardware and design, consistently setting trends within the market. This is evidenced by its development of new product lines such as Apple Watch and AirPods, which h

In [126]:
# --- Text2CypherRetriever Example ---
print("\n--- Text2CypherRetriever Example ---")
text2cypher_retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema="(:Chunk)-[]-(:Document)-[]-(:Cusip6)-[]-(:Company),(:Company)-[]-(:RiskFactor)(:Company)-[]-(:AssetManager)")

query = "Show me all companies managed by BlackRock and the risk factors they face."
cypher_query = text2cypher_retriever.get_search_results(query)
print("\n--- Text2Cypher Output ---")
print("Original Query:", query)
print("Generated Cypher:\n", cypher_query.metadata["cypher"])


--- Text2CypherRetriever Example ---

--- Text2Cypher Output ---
Original Query: Show me all companies managed by BlackRock and the risk factors they face.
Generated Cypher:
 MATCH (am:AssetManager {name: "BlackRock"})-[]-(c:Company)-[]-(rf:RiskFactor)
RETURN c.name AS Company, rf.name AS RiskFactor
